In [2]:
%%capture
"k"

In [1]:
%%capture
!pip install indic-nlp-library
!pip install morfessor

In [2]:
%%capture
!pip install googletrans==3.1.0a0

In [5]:
%%capture
!pip install translingual

In [3]:
import numpy as np
from tqdm.notebook import tqdm
from collections import OrderedDict
import pickle
from googletrans import Translator, constants
import time
import json
import os
import glob
from random import uniform

# Importing raw data

Data: https://nlp.stanford.edu/data/glove.6B.zip

In [8]:
from google.colab import drive
drive.mount('/content/drive')

project_path = "/content/drive/MyDrive/Colab Notebooks/Projects/PIE/"

with open(project_path + "Embeddings/english_embeddings.pickle", 'rb') as f:
    english_embeddings_dict = pickle.load(f)

Mounted at /content/drive


In [9]:
INDIC_NLP_RESOURCES=r"/content/drive/MyDrive/Colab Notebooks/Projects/PIE/indic_nlp_resources-master"

from indicnlp import common, loader
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()

from indicnlp.transliterate.unicode_transliterate import ItransTransliterator

The composition of the dictionary is as follows: corresponding to each word, there is an array of length matching the dimensions of the GloVe embeddings chosen.

# Translation

In [10]:
src_lang_dict = english_embeddings_dict
dest_language_code = 'hi'
dest_language = 'hindi'
dest_language_dict = OrderedDict()

In [6]:
!pip install deep-translator

In [8]:
deeptranslator = __import__("deep-translator")

ModuleNotFoundError: ignored

In [15]:
from deep_translator import GoogleTranslator

proxies_example = {
    "https": "34.195.196.27:8080",
    "http": "34.195.196.27:8080"
}
translated = GoogleTranslator(source='auto', target='de', proxies=proxies_example).translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


KeyError: ignored

In [ ]:
count = 0
#translator = Translator()
embeddings_path = project_path + 'Translated_Embeddings/' + dest_language
list_of_files = glob.glob(embeddings_path + "/*")
try:
    latest_count = int(max(list_of_files, key=os.path.getctime).replace(embeddings_path + '/' + dest_language + '_language_dict_' , "").replace('.json', ""))
except ValueError:
    latest_count = 0

src_lang_dict_items_list = list(src_lang_dict.items())

for k, v in tqdm(src_lang_dict_items_list[latest_count:]):
    translator = Translator()
    translated_word = translator.translate(k, dest=dest_language_code).text
    romanised_word = ItransTransliterator.to_itrans(translated_word, dest_language_code)
    dest_language_dict[translated_word] = v.tolist()
    count += 1
    #time.sleep(uniform(3, 4))
    if count % 30 == 0:
        print(translated_word, romanised_word)
        #time.sleep(uniform(10, 20))
    if count % 100 == 0:
        filename = embeddings_path + '/' + dest_language + '_language_dict_' + str(latest_count+1000) + '.json'
        with open(filename, 'w') as f:
            json.dump(dest_language_dict, f)
            #break
            time.sleep(60)
        dest_language_dict = OrderedDict()


In [ ]:
list(dest_language_dict.keys()).index('seine')

Concatenate translated embeddings

In [ ]:
"""
master_filename = project_path + 'Translated_Embeddings/' + dest_language + '/master/' + dest_language + '_language_dict' + '.json'
for count in [a for a in range(0, 40000) if a % 10 == 0]:
    filename = project_path + 'Translated_Embeddings/' + dest_language + '/' + dest_language + '_language_dict_' + str(count) + '.json'
    with open(filename, 'r') as j:
        contents = json.loads(j.read())
        with open(master_filename, 'w') as fp:
            fp.append('a')

In [ ]:
#da = json.loads(filename, object_pairs_hook=OrderedDict)

In [ ]:
'''
print(constants.LANGUAGES)
'''

In [ ]:
'''
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

corpus_size = file_len(file_path)
corpus_size
'''

In [ ]:
'''
def create_trans_dict_and_file(src_lang_file_path, dest_language_code, dest_language_path, dest_language_dict_pre):
  with open(src_lang_file_path, 'r', encoding="utf-8") as f_src:
    for line in tqdm(src_lang_file_path):
      count = 0
      word = line.split()[0]
      vector = np.asarray(line[1:], 'float32')
      translated_word = translator.translate(word, dest=dest_language_code).text
      translated_line = line.replace(word, translated_word)
      with open(dest_language_path, 'a') as f_dest:
        f_dest.write(translated_line)
      dest_language_dict_pre[word] = vector
      count += 1
      if count == 6:
        break
  f_src.close()
  return dest_language_dict
'''